## Code for automating the paraphrasing of specific texts given a file.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import shutil
import chime; chime.theme('mario')
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from datetime import date, datetime
from IPython.core.debugger import Pdb #Pdb().set_trace()


# Define your login credentials
username = "xxxxxxx@xxxxxx.xxxx"
password = "xxx"

In [2]:
all_files = sorted(os.listdir('data/'))[:8]

In [3]:
topics = [(theme
           .split('.')[0]
           .replace('C ', '')
           .replace('_', '')
           .replace(' 31','')
          ) for theme in all_files]
topics = dict(zip(range(len(topics)), topics))

#All files to be read
topics

{0: 'Entertainment',
 1: 'Finances and Business',
 2: 'Health and Wellbeing',
 3: 'Luxury',
 4: 'Personal Aesthetics',
 5: 'Social Awareness',
 6: 'Technology',
 7: 'Tourism'}

In [4]:
update_file = int(input(f"which file do you want to update?: \n{topics}\n\n"))
print(f"\n\nThe file <<{all_files[update_file]}>> will be updated.")

which file do you want to update?: 
{0: 'Entertainment', 1: 'Finances and Business', 2: 'Health and Wellbeing', 3: 'Luxury', 4: 'Personal Aesthetics', 5: 'Social Awareness', 6: 'Technology', 7: 'Tourism'}

7


The file <<C Tourism_.xlsx>> will be updated.


In [5]:
def read_not_published_data(file):
    df = pd.read_excel(f'data/{file}')
    not_published = df[df["published"]==0].reset_index(drop=True) #Not published yet
    not_published= not_published.rename(columns=str.lower)
    return not_published


not_published = read_not_published_data( all_files[update_file] )
# Self note, make warning for false not being the only value
not_published["published"].unique()

array([False])

In [6]:
not_published["paraphrased title"] = ["---"]*len(not_published)
not_published["paraphrased description"] = ["---"]*len(not_published)

In [7]:
def paraphrase_with_tool(not_published):
    # Creatting settings for log in 
    login_url = "https://xxxxxx.com/login"
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options)
    driver.implicitly_wait(60)

    # Initialization of session
    driver.get(login_url)
    cookies = driver.find_element(By.XPATH, '//*[@id="xxxxxxxxxxxx"]')
    ActionChains(driver).move_to_element(cookies).click(cookies).perform()
    usr_box = driver.find_element("name", 'username')
    pas_box = driver.find_element("name", 'password')
    login_button = driver.find_element(By.XPATH, '//*[@id="xxxxxxxxxxx')
    ActionChains(driver).move_to_element(usr_box).click(usr_box).send_keys(username).perform()
    ActionChains(driver).move_to_element(pas_box).click(pas_box).send_keys(password).perform()
    ActionChains(driver).move_to_element(login_button).click(login_button).perform()

    # Getting location of important buttons
    creative = driver.find_element(By.XPATH, '//*[@id="xxxxxxxxxxxxx')
    expand = driver.find_element(By.XPATH, '//*[@id="xxxxxxxxxxxx')
    paraphrased_box  = driver.find_element(By.XPATH, '//*[@id="xxxxxxxxxxxx"]')
    text_box = driver.find_element(By.CSS_SELECTOR, ".css-xxxxxxx")

    for i in range(len(not_published)):
        # Paraphrasing for case study title
        _control = paraphrased_box.text
        original_text_title = not_published["case study title"][i] 
        ActionChains(driver).move_to_element(creative).click(creative).perform()
        text_box.send_keys(original_text_title)
        # --  To paraphrase
        time.sleep(5) # required, otherwise webpage bugs
        ActionChains(driver).move_to_element(text_box).click(text_box).perform()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys(Keys.ENTER).perform()
        ActionChains(driver).move_to_element(paraphrased_box).click(paraphrased_box).perform()
        driver.find_element(By.CSS_SELECTOR, '.css-xxxxx') # sanity check: text finished processing
        time.sleep(5)
        _control = paraphrased_box.text
        (# -- Clean search box
            ActionChains(driver).move_to_element(text_box).click(text_box)
                .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
                .send_keys(Keys.DELETE).perform()
        )

        if len(_control) == 0:
            chime.warning()
            print("\n\n\n")
            print("Please, fix title. Afterwards, CLEAN THE TEXT BOX. When finished, press c")
            Pdb().set_trace()
            _control = paraphrased_box.text

        not_published.loc[i, "paraphrased title"] = _control

        (# -- Clean the paraphrased box
            ActionChains(driver).move_to_element(paraphrased_box).click(paraphrased_box)
                .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
                .send_keys(Keys.DELETE).perform()
        )    

        # Paraphrasong for case study description
        _control = paraphrased_box.text
        original_text_description = not_published["case study description"][i] 
        ActionChains(driver).move_to_element(expand).click(expand).perform()
        ActionChains(driver).move_to_element(text_box).click(text_box).perform()
        text_box.send_keys(original_text_description)
        # --  To paraphrase
        time.sleep(5) # required, otherwise webpage bugs
        ActionChains(driver).move_to_element(text_box).click(text_box).perform()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys(Keys.ENTER).perform()
        ActionChains(driver).move_to_element(paraphrased_box).click(paraphrased_box).perform()
        driver.find_element(By.CSS_SELECTOR, '.css-xxxxxxx') # sanity check: text finished processing
        time.sleep(7)
        _control = paraphrased_box.text
        (# -- Clean search box
            ActionChains(driver).move_to_element(text_box).click(text_box)
                .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
                .send_keys(Keys.DELETE).perform()
        )

        if len(_control) == 0:
            chime.warning()
            print("\n\n\n")
            print("Please, fix description. Afterwards, CLEAN THE TEXT BOX. When finished, press c")
            Pdb().set_trace()
            _control = paraphrased_box.text

        not_published.loc[i, "paraphrased description"] = _control

        (# -- Clean the paraphrased box
            ActionChains(driver).move_to_element(paraphrased_box).click(paraphrased_box)
                .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
                .send_keys(Keys.DELETE).perform()
        )
        return

Please, fix title. Afterwards, CLEAN THE TEXT BOX. When finished, press c
> /tmp/ipykernel_23328/427745214.py(49)<module>()
     47         print("Please, fix title. Afterwards, CLEAN THE TEXT BOX. When finished, press c")
     48         Pdb().set_trace()
---> 49         _control = paraphrased_box.text
     50 
     51     not_published.loc[i, "paraphrased title"] = _control

ipdb> original_text_title
'UNESCO, Airbnb and Mexico City partner to promote the city as a remote working hub'
ipdb> n
> /tmp/ipykernel_23328/427745214.py(51)<module>()
     49         _control = paraphrased_box.text
     50 
---> 51     not_published.loc[i, "paraphrased title"] = _control
     52 
     53     (# -- Clean the paraphrased box

ipdb> _control
'Mexico City is being promoted by UNESCO and Airbnb as a prime location for telecommuters. '
ipdb> c
